In [1]:
print('hi')

hi


In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense , LSTM ,Embedding,Dropout,BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import AdamW
from tensorflow.keras.losses import BinaryCrossentropy

In [5]:
model = Sequential()

model.add(Embedding(input_dim=50000,output_dim=300,input_length=252,trainable=True))
model.add(LSTM(256,activation='tanh',kernel_initializer='glorot_uniform',return_sequences=True))
model.add(BatchNormalization())
model.add(Dropout(0.15))

model.add(LSTM(128,activation='tanh',kernel_initializer='glorot_uniform',kernel_regularizer=l2(0.001),return_sequences=True))
model.add(BatchNormalization())
model.add(Dropout(0.10))

model.add(LSTM(64,activation='tanh',kernel_initializer='glorot_uniform',return_sequences=True))
model.add(BatchNormalization())
model.add(Dropout(0.05))

model.add(Dense(64,activation='relu',kernel_initializer='he_normal',kernel_regularizer=l2(0.001)))
model.add(BatchNormalization())
model.add(Dropout(0.05))

model.add(Dense(2,activation='sigmoid'))
model.compile(loss=BinaryCrossentropy(),optimizer=AdamW(learning_rate=1e-3),metrics=['accuracy'])
model.build(input_shape=(None,252))
model.summary()



c:\Users\risha\anaconda3\Lib\site-packages\keras\src\layers\core\embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)         │ (None, 252, 300)       │    15,000,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_6 (LSTM)                   │ (None, 252, 256)       │       570,368 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_8           │ (None, 252, 256)       │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 252, 256)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_7 (LSTM)                   │ (None, 252, 128)       │       197,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_9           │ (None, 252, 128)       │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_9 (Dropout)             │ (None, 252, 128)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_8 (LSTM)                   │ (None, 252, 64)        │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_10          │ (None, 252, 64)        │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_10 (Dropout)            │ (None, 252, 64)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 252, 64)        │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_11          │ (None, 252, 64)        │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_11 (Dropout)            │ (None, 252, 64)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 252, 2)         │           130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 15,823,234 (60.36 MB)

 Trainable params: 15,822,210 (60.36 MB)

 Non-trainable params: 1,024 (4.00 KB)

In [ ]:
import pandas as pd
df = pd.read_csv('embedded.csv',index_col=False)
X = df.drop(columns=['target'])
y = df['target']

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
history = model.fit(X,y,epochs=5,batch_size=64,validation_split=0.2,verbose=1,callbacks=[early_stopping])